In [1]:
pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 64.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=72daa4f63f17d49d1a09bab61b78c77bf0d3eaff2c749fbb9348e51b53f9f731
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()


data_schema = [ 
               StructField('id', StringType(), False),
               StructField('title', StringType(), True),
               StructField('price_old',DoubleType(), True),
               StructField('price_new', DoubleType(), True),
               StructField('link', StringType(), True),
               StructField('img', StringType(), True),
               
            ]

final_struc = StructType(fields = data_schema)

data = spark.read.csv(
    '/content/ring.csv',
    sep=';',
    header=True,
    schema=final_struc 
)

In [5]:
data1= data.withColumn('title', F.split(F.col('title'), ",").getItem(0))

output1 =(
    data1 
    .groupBy('title')
    .agg(  
        F.max('price_new').alias("max cost"),
        F.min('price_new').alias("min cost"))
    .orderBy('title')
)

In [8]:
output1.coalesce(1).write.mode("overwrite").format("csv").save("ring_max_min.csv")

In [17]:
df = data.withColumn('procent',(1 - data['price_new']/(data['price_old']))).orderBy(F.col("procent").desc())

In [ ]:
df.coalesce(1).write.mode("overwrite").format("csv").save("ring_top.csv")